In [38]:
import sys as sys 
sys.path.append('../')
from solver.hartree_fock import * 
import math

In [39]:
# Géométrie H2O (en Bohr)
OH_distance = 0.958 * 1.8897  # ~1.81 Bohr
angle_deg = 104.5
angle_rad = np.radians(angle_deg / 2)

atom_coordinates = [
    np.array([0.0, 0.0, 0.0]),  # Oxygène
    np.array([OH_distance * np.sin(angle_rad), 0.0, OH_distance * np.cos(angle_rad)]),  # H gauche
    np.array([-OH_distance * np.sin(angle_rad), 0.0, OH_distance * np.cos(angle_rad)])  # H droite
]

nuclei_charges = [8, 1, 1]
n_electrons = 10

# STO-3G coefficients et exposants (exemples simplifiés)
coeff_s = [0.1543289673, 0.5353281423, 0.4446345422]
exps_O_s = [130.70932, 23.808861, 6.4436083]
exps_H_s = [3.42525091, 0.62391373, 0.16885540]

coeff_p = [0.1543289673, 0.5353281423, 0.4446345422]
exps_O_p = [5.0331513, 1.1695961, 0.3803890]

# Créer les BasisFunctions
# Oxygène s
pg_list_O_s = [PrimGauss(atom_coordinates[0], exp, 0, 0, 0, normalise=True) for exp in exps_O_s]
bf_O_s = BasisFunction(pg_list_O_s, coeff_s)

# Oxygène p orbitals
pg_list_O_px = [PrimGauss(atom_coordinates[0], exp, 1, 0, 0, normalise=True) for exp in exps_O_p]
bf_O_px = BasisFunction(pg_list_O_px, coeff_p)

pg_list_O_py = [PrimGauss(atom_coordinates[0], exp, 0, 1, 0, normalise=True) for exp in exps_O_p]
bf_O_py = BasisFunction(pg_list_O_py, coeff_p)

pg_list_O_pz = [PrimGauss(atom_coordinates[0], exp, 0, 0, 1, normalise=True) for exp in exps_O_p]
bf_O_pz = BasisFunction(pg_list_O_pz, coeff_p)

# Hydrogène gauche s
pg_list_H1 = [PrimGauss(atom_coordinates[1], exp, 0, 0, 0, normalise=True) for exp in exps_H_s]
bf_H1 = BasisFunction(pg_list_H1, coeff_s)

# Hydrogène droite s
pg_list_H2 = [PrimGauss(atom_coordinates[2], exp, 0, 0, 0, normalise=True) for exp in exps_H_s]
bf_H2 = BasisFunction(pg_list_H2, coeff_s)

# Assemblage base
basis_functions = [bf_O_s, bf_O_px, bf_O_py, bf_O_pz, bf_H1, bf_H2]

# Initialisation et résolution RHF
rhf = RestrictedHartreeFock(
    basis_functions=basis_functions,
    atom_coordinates=atom_coordinates,
    nuclei_charges=nuclei_charges,
    n_electrons=n_electrons
)

electronic_energy = rhf.solve(tol=1e-8)
nuclear_repulsion = nuclear_nuclear_repulsion_energy(atom_coordinates, nuclei_charges)

print("Énergie électronique (Hartree) :", electronic_energy)
print("Énergie répulsion nucléaire (Hartree) :", nuclear_repulsion)
print("Énergie totale de la molécule (Hartree) :", electronic_energy + nuclear_repulsion)

HF loop converged at step  12
electronic energy :  -82.85953403272545
Énergie électronique (Hartree) : -82.85953403272545
Énergie répulsion nucléaire (Hartree) : 9.187460591319187
Énergie totale de la molécule (Hartree) : -73.67207344140627


In [40]:
import numpy as np
import plotly.graph_objects as go

# --- Tes fonctions pour generate_grid, evaluate_mo_on_grid (à adapter selon ta classe BasisFunction) ---

def generate_grid(atom_coords, spacing=0.3, padding=3.0):
    atom_coords = np.array(atom_coords)
    mins = atom_coords.min(axis=0) - padding
    maxs = atom_coords.max(axis=0) + padding

    x = np.arange(mins[0], maxs[0], spacing)
    y = np.arange(mins[1], maxs[1], spacing)
    z = np.arange(mins[2], maxs[2], spacing)
    X, Y, Z = np.meshgrid(x, y, z, indexing='ij')

    grid_points = np.vstack((X.ravel(), Y.ravel(), Z.ravel())).T
    shape = X.shape
    spacing_tuple = (x[1] - x[0], y[1] - y[0], z[1] - z[0])
    return grid_points, shape, spacing_tuple, X, Y, Z

def evaluate_mo_on_grid(grid_points, basis_functions, mo_coeffs, orbital_index):
    values = np.zeros(len(grid_points))
    for mu, bf in enumerate(basis_functions):
        phi_vals = np.array([bf(p) for p in grid_points])
        values += mo_coeffs[mu, orbital_index] * phi_vals
    return values

def plot_orbital_volume(values, X, Y, Z, atom_coords=None, atom_charges=None, iso_range=0.05):
    V = values.reshape(X.shape)

    fig = go.Figure()

    fig.add_trace(go.Volume(
        x=X.flatten(), y=Y.flatten(), z=Z.flatten(),
        value=V.flatten(),
        isomin=-iso_range,
        isomax=iso_range,
        opacity=0.15,
        surface_count=30,
        colorscale='RdBu',
        caps=dict(x_show=False, y_show=False, z_show=False)
    ))

    if atom_coords is not None and atom_charges is not None:
        for coord, Z in zip(atom_coords, atom_charges):
            fig.add_trace(go.Scatter3d(
                x=[coord[0]], y=[coord[1]], z=[coord[2]],
                mode='markers',
                marker=dict(
                    size=8 + 3 * Z,
                    color=Z,
                    colorscale='Viridis',
                    opacity=1.0,
                    line=dict(width=1, color='black')
                ),
                name=f'Noyau Z={Z}'
            ))

    fig.update_layout(
        scene=dict(
            xaxis_title='X (Bohr)',
            yaxis_title='Y (Bohr)',
            zaxis_title='Z (Bohr)',
            aspectmode='data'
        ),
        title='Orbital Molecular 3D avec positions atomiques',
        margin=dict(t=30, l=0, r=0, b=0)
    )
    fig.show()



In [43]:
# --- Exemple d’utilisation après calcul RHF ---

# Supposons que rhf est ton objet RestrictedHartreeFock avec les attributs :
# rhf.atom_coordinates, rhf.nuclei_charges, rhf.basis_functions, rhf.mos (coefficients MO)
# et que tu veux visualiser l'orbital occupé n°0 (le premier)

grid_points, shape, spacing, X, Y, Z = generate_grid(rhf.atom_coordinates, spacing=0.2, padding=3.0)
values = evaluate_mo_on_grid(grid_points, rhf.basis_functions, rhf.mos, orbital_index=5)  # Orbital 0

plot_orbital_volume(values, X, Y, Z,
                   atom_coords=rhf.atom_coordinates,
                   atom_charges=rhf.nuclei_charges,
                   iso_range=0.05)